In [1]:
import numpy as np
import pandas as pd

RNN Model Parameters

In [2]:
batchSize = 24
#lstmUnits = 64 # Not used
numClasses = 2  # Binary classification
#numClasses = 5
hiddenSize = 50

## Word Vectors

In [3]:
from pathlib import Path
import os.path
wordsList = np.load(os.path.join(str(Path.home()), '.kaggle/wordvectors/pretrained_glove/wordsList.npy'))
wordsList = wordsList.tolist() #Originally loaded as numpy array
wordsList = [word.decode('UTF-8') for word in wordsList] #Encode words as UTF-8
wordVectors = np.load(os.path.join(str(Path.home()), '.kaggle/wordvectors/pretrained_glove/wordVectors.npy'))

Word Vectors have dimension 50

In [4]:
embedding_dimension = 50

## Reviews

Load the Reviews

In [5]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /home/matt/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
review_df = pd.read_csv('~/.kaggle/datasets/snap/amazon-fine-food-reviews/Reviews.csv', encoding='utf8')
review_df = review_df.drop(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator', 'HelpfulnessDenominator', 'Time', 'Summary'], axis=1)

For development, just use 5000 rows for simplicity and speed

In [7]:
#numReviews = 20000

In [8]:
#review_df = review_df.loc[0:numReviews-1]

Lowercase, and then tokenize the reviews.  The tokens need to be lowercase for the embedding lookup.

In [9]:
review_df['Tokens'] = review_df['Text'].apply(lambda text: word_tokenize(text.lower()))

Create the array of input sentences converted to word IDs

In [10]:
maxSeqLength = 250  # Determined by EDA

In [11]:
word_ids = np.zeros((review_df.shape[0], maxSeqLength), dtype='int32')

Convert words to word IDs and store in word_ids

In [12]:
for sentence_index, row in review_df.iterrows():
    
    word_index = 0
    
    for word in row['Tokens']:

        try:
            word_ids[sentence_index][word_index] = wordsList.index(word)
        except ValueError:
            word_ids[sentence_index][word_index] = 399999 #Vector for unkown words
        
        word_index = word_index + 1

        if word_index == maxSeqLength:
            break

Create functions to get the train and test batches

In [13]:
# Train with 70%, test with 30%
train_percent = 0.7

# Get indicies of the rows in the dataframe for training and testing
train_lower_index = 0
train_upper_index = round(len(review_df)*train_percent)
test_lower_index = train_upper_index+1
test_upper_index = len(review_df)-1

In [14]:
# Global variables
next_train_index = train_lower_index
next_test_index = test_lower_index

TODO: Look into replacing these functions with Tensorflow's data iterators

In [15]:
def getTrainBatch(dataset):
    global next_train_index
    
    data_len = len(dataset)
    
    word_batch = np.zeros([batchSize, maxSeqLength])
    label_batch = np.zeros([batchSize, numClasses])
    
    continue_iterating = True
    
    if ( (next_train_index + batchSize) > data_len ):
        continue_iterating = False
    
    next_batch = min(batchSize, (data_len - next_train_index))

    for i in range(next_batch):
        word_batch[i] = word_ids[next_train_index + i]
        
# Non-binary classification
#        label_batch[i] = np.zeros(numClasses)
#        label_batch[i][review_df.iloc[next_train_index + i].Score - 1] = 1

# Binary classification
        if review_df.iloc[next_train_index + i].Score >= 3:
            label_batch[i] = [0, 1]
        else:
            label_batch[i] = [1, 0]
        
    next_train_index = next_train_index + next_batch
    
    return continue_iterating, word_batch, label_batch

def getTestBatch(dataset):
    global next_test_index
    
    data_len = len(dataset)
    
    word_batch = np.zeros([batchSize, maxSeqLength])
    label_batch = np.zeros([batchSize, numClasses])
    
    continue_iterating = True
    
    if ( (next_test_index + batchSize) > data_len ):
        continue_iterating = False
    
    next_batch = min(batchSize, (data_len - next_test_index))

    for i in range(next_batch):
        word_batch[i] = word_ids[next_test_index + i]
        
# Non-binary classification
#        label_batch[i] = np.zeros(numClasses)
#        label_batch[i][review_df.iloc[next_test_index + i].Score - 1] = 1

# Binary classification
        if review_df.iloc[next_test_index + i].Score >= 3:
            label_batch[i] = [0, 1]
        else:
            label_batch[i] = [1, 0]

    current_test_index = next_test_index
    next_test_index = next_test_index + next_batch
    
    return continue_iterating, current_test_index, word_batch, label_batch

def resetBatch():
    global next_train_index
    global next_test_index
    
    next_train_index = train_lower_index
    next_test_index = test_lower_index

Reset the batch iterators

In [16]:
resetBatch()

## Tensorflow Model

Hyper Parameters

In [17]:
learning_rate = 0.01
dropout_keep_prob = 0.75

Model

In [18]:
import tensorflow as tf
tf.reset_default_graph()

labels = tf.placeholder(tf.float32, [batchSize, numClasses])
input_data = tf.placeholder(tf.int32, [batchSize, maxSeqLength])

data = tf.Variable(tf.zeros([batchSize, maxSeqLength, embedding_dimension]), dtype=tf.float32)
data = tf.nn.embedding_lookup(wordVectors, input_data)

lstmCell = tf.contrib.rnn.BasicLSTMCell(hiddenSize)
lstmCell = tf.contrib.rnn.DropoutWrapper(cell=lstmCell, output_keep_prob=dropout_keep_prob)
rnn_out, _ = tf.nn.dynamic_rnn(lstmCell, data, dtype=tf.float32)

W_out = tf.Variable(tf.truncated_normal([hiddenSize, numClasses]), dtype=tf.float32)
b_out = tf.Variable(tf.constant(0.1, shape=[numClasses]), dtype=tf.float32)

# Get the output of the last RNN cell
rnn_out = tf.transpose(rnn_out, [1, 0, 2])
last_cell_out = tf.gather(rnn_out, int(rnn_out.get_shape()[0]) - 1)

# Calculate logits
logits = (tf.matmul(last_cell_out, W_out) + b_out)

# Calculate prediction and accuracy
prediction = tf.argmax(logits,1)
correctPred = tf.equal(prediction, tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

# Loss function and optimizer
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



## Train Model

In [19]:
import datetime

def TrainModel(session, writer):
    
    i = 0
    start_time = datetime.datetime.now()

    while True:
    
        # Next Batch of reviews
        continue_iterating, nextBatch, nextBatchLabels = getTrainBatch(review_df)
    
        feed_dict_ = {
            input_data: nextBatch,
            labels: nextBatchLabels
        }

        loss_, _ = session.run([loss, optimizer], feed_dict=feed_dict_)

        #Write summary to Tensorboard
        if (i % 10 == 0):
            summary = session.run(merged, {input_data: nextBatch, labels: nextBatchLabels})
            writer.add_summary(summary, i)

        if (i % 500 == 0):
            print("Loss is: ", loss_, ", ", (datetime.datetime.now() - start_time).seconds, " seconds")

        i = i + 1

        if not continue_iterating:
            print("Loss is: ", loss_, "\n")
            break

## Test Model

In [20]:
def TestModel(session):
    
    while True:
    
        continue_iterating_, current_test_index_, nextBatch_, nextBatchLabels_ = getTestBatch(review_df);

        feed_dict = {
            input_data: nextBatch_,
            labels: nextBatchLabels_
        }

        #loss_ = sess.run(loss, feed_dict)
        #print("Loss is: ", loss_, "\n")

        accuracy_ = sess.run(accuracy, feed_dict)

        if accuracy_ < 1.0:
            print("Accuracy for this batch: ", accuracy_ * 100)
            
            predictions_ = sess.run(prediction, feed_dict)
            
            for index in range(len(predictions_)):

                if predictions_[index] != np.argmax(nextBatchLabels_[index]):
                    print("Sentence mispredicted: ", current_test_index_+index)
        
        if not continue_iterating_:
            break

## Train and Test

In [21]:
import datetime

with tf.Session() as sess:
    #saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    
    tf.summary.scalar('Loss', loss)
    tf.summary.scalar('Accuracy', accuracy)
    merged = tf.summary.merge_all()
    logdir = "tensorboard/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "/"
    writer = tf.summary.FileWriter(logdir, sess.graph)
    
    resetBatch()
    
    # Train model
    TrainModel(sess, writer)
    
    # Test model
    TestModel(sess)
    
    writer.close()

Loss is:  0.69316006 ,  1  seconds
Loss is:  0.718374 ,  34  seconds
Loss is:  0.29966533 ,  67  seconds
Loss is:  0.2715236 ,  100  seconds
Loss is:  0.45382857 ,  133  seconds
Loss is:  0.5392087 ,  166  seconds
Loss is:  0.15529868 ,  199  seconds
Loss is:  0.21419913 ,  232  seconds
Loss is:  0.1689473 ,  266  seconds
Loss is:  0.22783695 ,  299  seconds
Loss is:  0.3096428 ,  332  seconds
Loss is:  0.374352 ,  365  seconds
Loss is:  0.2099585 ,  398  seconds
Loss is:  0.6411818 ,  431  seconds
Loss is:  0.29053977 ,  464  seconds
Loss is:  0.13805263 ,  497  seconds
Loss is:  0.22724028 ,  530  seconds
Loss is:  0.5870971 ,  564  seconds
Loss is:  0.9628207 ,  597  seconds
Loss is:  0.51567596 ,  630  seconds
Loss is:  0.39940718 ,  663  seconds
Loss is:  0.4527432 ,  696  seconds
Loss is:  0.13601266 ,  729  seconds
Loss is:  0.3838345 ,  762  seconds
Loss is:  0.18637775 ,  795  seconds
Loss is:  0.38228944 ,  828  seconds
Loss is:  0.13718736 ,  861  seconds
Loss is:  0.4606441

Sentence mispredicted:  399179
Sentence mispredicted:  399185
Accuracy for this batch:  79.16666865348816
Sentence mispredicted:  399208
Sentence mispredicted:  399209
Sentence mispredicted:  399210
Sentence mispredicted:  399211
Sentence mispredicted:  399214
Accuracy for this batch:  66.66666865348816
Sentence mispredicted:  399219
Sentence mispredicted:  399220
Sentence mispredicted:  399221
Sentence mispredicted:  399225
Sentence mispredicted:  399229
Sentence mispredicted:  399231
Sentence mispredicted:  399237
Sentence mispredicted:  399238
Accuracy for this batch:  91.66666865348816
Sentence mispredicted:  399239
Sentence mispredicted:  399249
Accuracy for this batch:  95.83333134651184
Sentence mispredicted:  399271
Accuracy for this batch:  66.66666865348816
Sentence mispredicted:  399287
Sentence mispredicted:  399288
Sentence mispredicted:  399289
Sentence mispredicted:  399290
Sentence mispredicted:  399301
Sentence mispredicted:  399303
Sentence mispredicted:  399304
Sente

Accuracy for this batch:  83.33333134651184
Sentence mispredicted:  400908
Sentence mispredicted:  400910
Sentence mispredicted:  400912
Sentence mispredicted:  400913
Accuracy for this batch:  70.83333134651184
Sentence mispredicted:  400921
Sentence mispredicted:  400931
Sentence mispredicted:  400935
Sentence mispredicted:  400937
Sentence mispredicted:  400938
Sentence mispredicted:  400941
Sentence mispredicted:  400942
Accuracy for this batch:  25.0
Sentence mispredicted:  400945
Sentence mispredicted:  400946
Sentence mispredicted:  400948
Sentence mispredicted:  400949
Sentence mispredicted:  400951
Sentence mispredicted:  400952
Sentence mispredicted:  400953
Sentence mispredicted:  400954
Sentence mispredicted:  400957
Sentence mispredicted:  400958
Sentence mispredicted:  400959
Sentence mispredicted:  400960
Sentence mispredicted:  400961
Sentence mispredicted:  400962
Sentence mispredicted:  400963
Sentence mispredicted:  400964
Sentence mispredicted:  400965
Sentence misp

Sentence mispredicted:  402243
Sentence mispredicted:  402249
Accuracy for this batch:  83.33333134651184
Sentence mispredicted:  402264
Sentence mispredicted:  402267
Sentence mispredicted:  402275
Sentence mispredicted:  402280
Accuracy for this batch:  91.66666865348816
Sentence mispredicted:  402287
Sentence mispredicted:  402288
Accuracy for this batch:  75.0
Sentence mispredicted:  402342
Sentence mispredicted:  402347
Sentence mispredicted:  402348
Sentence mispredicted:  402355
Sentence mispredicted:  402356
Sentence mispredicted:  402358
Accuracy for this batch:  79.16666865348816
Sentence mispredicted:  402362
Sentence mispredicted:  402363
Sentence mispredicted:  402367
Sentence mispredicted:  402370
Sentence mispredicted:  402379
Accuracy for this batch:  66.66666865348816
Sentence mispredicted:  402384
Sentence mispredicted:  402385
Sentence mispredicted:  402386
Sentence mispredicted:  402389
Sentence mispredicted:  402391
Sentence mispredicted:  402400
Sentence mispredic

Accuracy for this batch:  91.66666865348816
Sentence mispredicted:  403616
Sentence mispredicted:  403621
Accuracy for this batch:  66.66666865348816
Sentence mispredicted:  403631
Sentence mispredicted:  403632
Sentence mispredicted:  403633
Sentence mispredicted:  403635
Sentence mispredicted:  403636
Sentence mispredicted:  403637
Sentence mispredicted:  403639
Sentence mispredicted:  403641
Accuracy for this batch:  95.83333134651184
Sentence mispredicted:  403675
Accuracy for this batch:  87.5
Sentence mispredicted:  403683
Sentence mispredicted:  403684
Sentence mispredicted:  403690
Accuracy for this batch:  91.66666865348816
Sentence mispredicted:  403716
Sentence mispredicted:  403718
Accuracy for this batch:  83.33333134651184
Sentence mispredicted:  403737
Sentence mispredicted:  403738
Sentence mispredicted:  403743
Sentence mispredicted:  403750
Accuracy for this batch:  66.66666865348816
Sentence mispredicted:  403752
Sentence mispredicted:  403755
Sentence mispredicted: 

...

Accuracy for this batch:  91.66666865348816
Sentence mispredicted:  560401
Sentence mispredicted:  560416
Accuracy for this batch:  87.5
Sentence mispredicted:  560437
Sentence mispredicted:  560439
Sentence mispredicted:  560441
Accuracy for this batch:  95.83333134651184
Sentence mispredicted:  560473
Accuracy for this batch:  83.33333134651184
Sentence mispredicted:  560502
Sentence mispredicted:  560508
Sentence mispredicted:  560509
Sentence mispredicted:  560516
Accuracy for this batch:  66.66666865348816
Sentence mispredicted:  560521
Sentence mispredicted:  560522
Sentence mispredicted:  560524
Sentence mispredicted:  560526
Sentence mispredicted:  560537
Sentence mispredicted:  560540
Sentence mispredicted:  560541
Sentence mispredicted:  560542
Accuracy for this batch:  58.33333134651184
Sentence mispredicted:  560543
Sentence mispredicted:  560545
Sentence mispredicted:  560546
Sentence mispredicted:  560547
Sentence mispredicted:  560549
Sentence mispredicted:  560550
Sente

Sentence mispredicted:  561747
Sentence mispredicted:  561749
Sentence mispredicted:  561754
Accuracy for this batch:  91.66666865348816
Sentence mispredicted:  561801
Sentence mispredicted:  561807
Accuracy for this batch:  75.0
Sentence mispredicted:  561815
Sentence mispredicted:  561822
Sentence mispredicted:  561823
Sentence mispredicted:  561824
Sentence mispredicted:  561830
Sentence mispredicted:  561836
Accuracy for this batch:  95.83333134651184
Sentence mispredicted:  561843
Accuracy for this batch:  87.5
Sentence mispredicted:  561868
Sentence mispredicted:  561870
Sentence mispredicted:  561877
Accuracy for this batch:  91.66666865348816
Sentence mispredicted:  561909
Sentence mispredicted:  561910
Accuracy for this batch:  83.33333134651184
Sentence mispredicted:  561913
Sentence mispredicted:  561914
Sentence mispredicted:  561921
Sentence mispredicted:  561924
Accuracy for this batch:  87.5
Sentence mispredicted:  561937
Sentence mispredicted:  561940
Sentence mispredic

Accuracy for this batch:  75.0
Sentence mispredicted:  563999
Sentence mispredicted:  564000
Sentence mispredicted:  564001
Sentence mispredicted:  564002
Sentence mispredicted:  564003
Sentence mispredicted:  564004
Accuracy for this batch:  87.5
Sentence mispredicted:  564023
Sentence mispredicted:  564028
Sentence mispredicted:  564033
Accuracy for this batch:  66.66666865348816
Sentence mispredicted:  564047
Sentence mispredicted:  564049
Sentence mispredicted:  564059
Sentence mispredicted:  564062
Sentence mispredicted:  564063
Sentence mispredicted:  564064
Sentence mispredicted:  564067
Sentence mispredicted:  564069
Accuracy for this batch:  87.5
Sentence mispredicted:  564083
Sentence mispredicted:  564086
Sentence mispredicted:  564089
Accuracy for this batch:  95.83333134651184
Sentence mispredicted:  564113
Accuracy for this batch:  83.33333134651184
Sentence mispredicted:  564129
Sentence mispredicted:  564131
Sentence mispredicted:  564137
Sentence mispredicted:  564139


Sentence mispredicted:  565834
Sentence mispredicted:  565836
Sentence mispredicted:  565837
Sentence mispredicted:  565839
Sentence mispredicted:  565841
Sentence mispredicted:  565843
Sentence mispredicted:  565844
Sentence mispredicted:  565845
Accuracy for this batch:  79.16666865348816
Sentence mispredicted:  565852
Sentence mispredicted:  565853
Sentence mispredicted:  565857
Sentence mispredicted:  565861
Sentence mispredicted:  565866
Accuracy for this batch:  87.5
Sentence mispredicted:  565874
Sentence mispredicted:  565875
Sentence mispredicted:  565890
Accuracy for this batch:  70.83333134651184
Sentence mispredicted:  565897
Sentence mispredicted:  565898
Sentence mispredicted:  565911
Sentence mispredicted:  565912
Sentence mispredicted:  565913
Sentence mispredicted:  565916
Sentence mispredicted:  565918
Accuracy for this batch:  95.83333134651184
Sentence mispredicted:  565935
Accuracy for this batch:  83.33333134651184
Sentence mispredicted:  565946
Sentence mispredic

Accuracy for this batch:  70.83333134651184
Sentence mispredicted:  567432
Sentence mispredicted:  567435
Sentence mispredicted:  567441
Sentence mispredicted:  567442
Sentence mispredicted:  567444
Sentence mispredicted:  567445
Sentence mispredicted:  567446
Accuracy for this batch:  75.0
Sentence mispredicted:  567455
Sentence mispredicted:  567458
Sentence mispredicted:  567460
Sentence mispredicted:  567462
Sentence mispredicted:  567465
Sentence mispredicted:  567466
Accuracy for this batch:  95.83333134651184
Sentence mispredicted:  567490
Accuracy for this batch:  79.16666865348816
Sentence mispredicted:  567503
Sentence mispredicted:  567504
Sentence mispredicted:  567505
Sentence mispredicted:  567517
Sentence mispredicted:  567519
Accuracy for this batch:  66.66666865348816
Sentence mispredicted:  567530
Sentence mispredicted:  567531
Sentence mispredicted:  567534
Sentence mispredicted:  567535
Sentence mispredicted:  567538
Sentence mispredicted:  567541
Sentence mispredic